<h1 align=center><font size = 5><h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1></font></h1>

## 0. Introduction
This Notebook is for Coursera course, Applied Data Science Capstone, week 3 assignment.

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [113]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## 1. Collect and collate data from Wikipedia

In [197]:
#read and copy the website data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.text,'lxml')

#find the content corresponding to the table
table = soup.find('table')

#get the table headers col_names
headers = table.find_all('th')
col_names = []
for i in range(len(headers)):
    col_names.append(header[i].text.rstrip())
print('There are ', len(col_names), ' columns in the table, headers are ')
print(col_names)

#get the detailed table contents
contents = []
for row in table.find_all('td'):
    contents.append(row.text.rstrip())
print('\nThere are ', int(len(contents)/3), ' rows in the table.')

There are  3  columns in the table, headers are 
['Postal code', 'Borough', 'Neighborhood']

There are  180  rows in the table.


##### Structure the table contents in a dataframe
1. Ignore cells with a borough that is 'Not assigned'.
2. Neighborhoods sharing the same postal code will be combined into one row with a comma for seperation.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [277]:
#raw table from Wikipedia
df_raw = pd.DataFrame(columns = col_names, index=range(180))
for i in range(int(len(contents)/3)):
    df_raw['Postal code'][i] = contents[3*i + 0]
    df_raw['Borough'][i] = contents[3*i + 1]
    df_raw['Neighborhood'][i] = contents[3*i + 2]

#remove 'Not assigned'
df = df_raw[df_raw.Borough !='Not assigned']

#seperate with comma
df.reset_index(drop= True, inplace=True)

for i in range(df.shape[0]):
    df['Neighborhood'][i] = df['Neighborhood'][i].replace(" / ", ', ')
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [279]:
#print the number of rows of the dataframe
df.shape

(103, 3)